# GAN implementation by tensorflow

In [1]:
import tensorflow as tf
import os
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

# utility


In [2]:
def save_metrics(path, metrics, epoch=None):
    if not os.path.isdir(path):
        os.makedirs(path)

    plt.figure(figsize=(10,8))
    plt.plot(metrics["d_loss"], label="discriminative loss", color="b")
    plt.legend()
    plt.savefig(os.path.join(path, "dloss" + str(epoch) + ".png"))
    plt.close()

    plt.figure(figsize=(10,8))
    plt.plot(metrics["g_loss"], label="generative loss", color="r")
    plt.legend()
    plt.savefig(os.path.join(path, "g_loss" + str(epoch) + ".png"))
    plt.close()

    plt.figure(figsize=(10,8))
    plt.plot(metrics["g_loss"], label="generative loss", color="r")
    plt.plot(metrics["d_loss"], label="discriminative loss", color="b")
    plt.legend()
    plt.savefig(os.path.join(path, "both_loss" + str(epoch) + ".png"))
    plt.close()

In [3]:
# save generated images
def save_imgs(path, images, plot_dim=(5,12), size=(24,10), epoch=None):
    if not os.path.isdir(path):
        os.makedirs(path)

    examples = plot_dim[0]*plot_dim[1]

    # 表示
    fig = plt.figure(figsize=size)
    for i in range(examples):
        if i > images.shape[0]-1:
            continue
        plt.subplot(plot_dim[0], plot_dim[1], i+1)
        img = images[i, :]
        img = img.reshape((128, 128, 3))
        plt.tight_layout()
        plt.imshow(img)
        plt.axis("off")
    plt.subplots_adjust(wspace=0.1, hspace=0.1)
    plt.savefig(os.path.join(path, str(epoch) + ".png"))
    plt.close()


# モデル

In [4]:
class Generator:
    def __init__(self, batch_size):
        self.reuse = False
        self.z_dim = 128
        self.s_size = 8
        self.depths = [1024, 512, 256, 128, 3]

        self.num_res_blocks = 16
        self.num_pixel_CNN_blocks = 3

        self.batch_size = batch_size
        
    def __call__(self, z, training=False):
        with tf.variable_scope('g', reuse=self.reuse):
            # reshape from inputs
            with tf.variable_scope('fc0'):
                z0 = tf.reshape(z, [-1, self.z_dim])

                fc0 = tf.layers.dense(z0, 64*16*16, kernel_initializer=tf.truncated_normal_initializer(0.0, 0.02))            
                fc0 = tf.layers.batch_normalization(fc0, training=training)
                fc0 = tf.nn.relu(fc0)
                fc0 = tf.reshape(fc0, [-1,16,16,64])

            assert fc0.get_shape().as_list()[1:] == [16,16,64]
            
            layers = []
            layers.append(fc0)
            
            for i in range(int(self.num_res_blocks)):
                with tf.variable_scope('res_%d' % (i+1)):
                    res = tf.layers.conv2d(layers[-1], 64, [3,3], [1,1], padding="SAME", kernel_initializer=tf.truncated_normal_initializer(0.0, 0.02)) # 8x8
                    res = tf.layers.batch_normalization(res, training=training)
                    res = tf.nn.relu(res)

                    res = tf.layers.conv2d(res, 64, [3,3], [1,1], padding="SAME", kernel_initializer=tf.truncated_normal_initializer(0.0, 0.02))
                    res = tf.layers.batch_normalization(res, training=training)
                    res = layers[-1] + res
                    layers.append(res)
            
            assert layers[-1].get_shape().as_list()[1:] == [16,16,64]
            
            with tf.variable_scope('conv17'):
                conv17 = tf.layers.conv2d(layers[-1], 64, [3,3], [1,1], padding="SAME", kernel_initializer=tf.truncated_normal_initializer(0.0, 0.02)) # 8x8
                conv17 = tf.layers.batch_normalization(conv17, training=training)
                conv17 = tf.nn.relu(conv17)
                conv17 = layers[0] + conv17
                layers.append(conv17)

            assert layers[-1].get_shape().as_list()[1:] == [16, 16, 64]

            def pixel_shuffle_layer(x, r, n_split):
                def PS(x, r):
                    bs, a, b, c = x.get_shape().as_list()
                    bs = tf.shape(x)[0]
                    x = tf.reshape(x, (bs, a, b, r, r))
                    x = tf.transpose(x, (0, 1, 2, 4, 3))
                    x = tf.split(x, a, 1)
                    x = tf.concat([tf.squeeze(x_) for x_ in x], 2)
                    x = tf.split(x, b, 1)
                    x = tf.concat([tf.squeeze(x_) for x_ in x], 2)
                    return tf.reshape(x, (bs, a*r, b*r, 1))

                xc = tf.split(x, n_split, 3)
                return tf.concat([PS(x_, r) for x_ in xc], 3)

            for i in range(int(self.num_pixel_CNN_blocks)):
                with tf.variable_scope('pixel_CNN_%d' % (i+1)):
                    ps = tf.layers.conv2d(layers[-1], 256, [3,3], [1,1], padding="SAME", kernel_initializer=tf.truncated_normal_initializer(0.0, 0.02)) # 8x8
                    #print(ps.get_shape())
                    ps = pixel_shuffle_layer(ps, 2, 64)
                    ps = tf.layers.batch_normalization(ps, training=training)
                    ps = tf.nn.relu(ps)
                    layers.append(ps)
                    #print(ps.get_shape())

            assert layers[-1].get_shape().as_list()[1:] == [128, 128, 64]
                    
            with tf.variable_scope('output'):
                output = tf.layers.conv2d(layers[-1], 3, [9,9], [1,1], padding="SAME", kernel_initializer=tf.truncated_normal_initializer(0.0, 0.02)) # 8x8
                #output = tf.nn.tanh(output)
                output = tf.nn.sigmoid(output)

            assert output.get_shape().as_list()[1:] == [128, 128, 3]

        self.reuse = True
        self.variables = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='g')
        return output

In [5]:
class Discriminator:
    def __init__(self):
        self.reuse = False
        self.X_dim = 128*128*3
        self.s_size = 8
        self.depths = [3, 32, 128, 256, 512]

    def __call__(self, x, training=False, name=''):
        def leaky_relu(x, leak=0.2, name='outputs'):
            return tf.maximum(x, x * leak, name=name)

        with tf.name_scope('d' + name), tf.variable_scope('d', reuse=self.reuse):
            # convolution x 4
            x = tf.reshape(x, [-1, 128, 128, 3])
            with tf.variable_scope('conv1'):
                conv1 = tf.layers.conv2d(x, 32, [4, 4], [2 ,2], padding="SAME", kernel_initializer=tf.truncated_normal_initializer(0.0, 0.02))
                conv1 = leaky_relu(conv1)

            with tf.variable_scope('res1'):
                res1 = tf.layers.conv2d(conv1, 32, [3,3], [1,1], padding="SAME", kernel_initializer=tf.truncated_normal_initializer(0.0, 0.02))
                res1 = leaky_relu(res1)
                res1 = tf.layers.conv2d(res1, 32, [3,3], [1,1], padding="SAME", kernel_initializer=tf.truncated_normal_initializer(0.0, 0.02))
                res1 = res1 + conv1
                res1 = leaky_relu(res1)

            with tf.variable_scope('res2'):
                res2 = tf.layers.conv2d(res1, 32, [3,3], [1,1], padding="SAME", kernel_initializer=tf.truncated_normal_initializer(0.0, 0.02))
                res2 = leaky_relu(res2)
                res2 = tf.layers.conv2d(res2, 32, [3,3], [1,1], padding="SAME", kernel_initializer=tf.truncated_normal_initializer(0.0, 0.02))
                res2 = res2 + res1
                res2 = leaky_relu(res2)

            with tf.variable_scope('conv2'):
                conv2 = tf.layers.conv2d(res2, 64, [4, 4], [2 ,2], padding="SAME", kernel_initializer=tf.truncated_normal_initializer(0.0, 0.02))
                conv2 = leaky_relu(conv2)

            with tf.variable_scope('res3'):
                res3 = tf.layers.conv2d(conv2, 64, [3,3], [1,1], padding="SAME", kernel_initializer=tf.truncated_normal_initializer(0.0, 0.02))
                res3 = leaky_relu(res3)
                res3 = tf.layers.conv2d(res3, 64, [3,3], [1,1], padding="SAME", kernel_initializer=tf.truncated_normal_initializer(0.0, 0.02))
                res3 = leaky_relu(res3)
                res3 = res3 + conv2
                res3 = leaky_relu(res3)

            with tf.variable_scope('res4'):
                res4 = tf.layers.conv2d(res3, 64, [3,3], [1,1], padding="SAME", kernel_initializer=tf.truncated_normal_initializer(0.0, 0.02))
                res4 = leaky_relu(res4)
                res4 = tf.layers.conv2d(res4, 64, [3,3], [1,1], padding="SAME", kernel_initializer=tf.truncated_normal_initializer(0.0, 0.02))
                res4 = leaky_relu(res4)
                res4 = res4 + res3
                res4 = leaky_relu(res4)

            with tf.variable_scope('conv3'):
                conv3 = tf.layers.conv2d(res4, 128, [4, 4], [2 ,2], padding="SAME", kernel_initializer=tf.truncated_normal_initializer(0.0, 0.02))
                conv3 = leaky_relu(conv3)

            num_res_itr = 3
            layers = []
            layers.append(conv3)
            
            depth = [128, 256, 512, 1024]
            for i in range(int(num_res_itr)):
                with tf.variable_scope('res_%d_1' % (i+1+4)):
                    res = tf.layers.conv2d(layers[-1], depth[i], [3,3], [1,1], padding="SAME", kernel_initializer=tf.truncated_normal_initializer(0.0, 0.02))
                    res = leaky_relu(res)
                    res = tf.layers.conv2d(res, depth[i], [3,3], [1,1], padding="SAME", kernel_initializer=tf.truncated_normal_initializer(0.0, 0.02))
                    res = leaky_relu(res)
                    res = layers[-1] + res
                    res = leaky_relu(res)
                layers.append(res)

                with tf.variable_scope('res_%d_2' % (i+1+4)):
                    res = tf.layers.conv2d(layers[-1], depth[i], [3,3], [1,1], padding="SAME", kernel_initializer=tf.truncated_normal_initializer(0.0, 0.02))
                    res = leaky_relu(res)
                    res = tf.layers.conv2d(res, depth[i], [3,3], [1,1], padding="SAME", kernel_initializer=tf.truncated_normal_initializer(0.0, 0.02))
                    res = leaky_relu(res)
                    res = layers[-1] + res
                    res = leaky_relu(res)

                conv = tf.layers.conv2d(res, depth[i+1], [4, 4], [2 ,2], padding="SAME", kernel_initializer=tf.truncated_normal_initializer(0.0, 0.02))
                conv = leaky_relu(conv) 
                layers.append(conv)

                
            output = tf.layers.dense(layers[-1], 1, kernel_initializer=tf.truncated_normal_initializer(0.0, 0.02)) 
            #output = tf.nn.sigmoid(output)
        self.reuse = True
        self.variables = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='d')
        return output

In [6]:
# X_mb, Y_mbを収集
def extractXimg(path, batch_size, img_size):
    imgs = os.listdir(path)
    rand_id = np.random.randint(0, len(imgs), size=batch_size)
    X_mb = np.zeros((1, img_size, img_size, 3))
    for i in range(batch_size):
        img = Image.open(path+"/"+imgs[rand_id[i]])
        background = Image.new("RGB", img.size, (255,255,255))
        background.paste(img, mask=img.split()[3])
        img_np = np.asarray(background)
        X_mb = np.vstack((X_mb, img_np[np.newaxis, :]))
        img.close()
        background.close()
    X_mb = X_mb[1:,:]
    return X_mb

In [ ]:
class DRAGAN:
    def __init__(self):
        self.batch_size = 16
        self.g = Generator(self.batch_size)
        self.d = Discriminator()
        self.z_dim = 128

        self.img_size = 128
        
        self.epochs = 500000
        self.epoch_saveMetrics = 100
        self.epoch_saveSampleImg = 100
        self.epoch_saveParameter = 10000
        self.losses = {"d_loss":[], "g_loss":[]}

        self.Z = tf.placeholder(tf.float32, [None, self.z_dim])
        self.X_tr = tf.placeholder(tf.float32, [None, self.img_size*self.img_size*3])
        self.X_p = tf.placeholder(tf.float32, [None, self.img_size*self.img_size*3])

        self.X_g = self.g(self.Z, training=True)
        self.Y_tr = self.d(self.X_tr, training=True)
        self.Y_g = self.d(self.X_g, training=True)

        self.noise = np.random.uniform(-1, 1, size=[self.batch_size, self.z_dim]).astype(np.float32)
        self.L_g, self.L_d = self.loss()
        self.learning_rate = tf.placeholder("float", [])

    def d_loss_calc(self, gradient_penalty):
        lambda_ = 10

        d_loss_train = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=self.Y_tr, labels=tf.ones_like(self.Y_tr)))
        d_loss_gen = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=self.Y_g, labels=tf.zeros_like(self.Y_g)))
        loss = d_loss_train + d_loss_gen
        loss = loss + lambda_*gradient_penalty
        return loss

    def g_loss_calc(self):
        loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=self.Y_g, labels=tf.ones_like(self.Y_g)))
        return loss

    def loss(self):
        #z = tf.random_uniform([self.batch_size, self.z_dim], minval=-1.0, maxval=1.0)

        alpha = tf.random_uniform(shape=[self.batch_size,1], minval=0., maxval=1.)
        #print(alpha.get_shape())
        #print(self.X_p.get_shape())
        diff = self.X_p - self.X_tr
        interpolates = self.X_tr + (alpha*diff)
        gradients = tf.gradients(self.d(interpolates), [interpolates])[0]
        slopes = tf.sqrt(tf.reduce_sum(tf.square(gradients), reduction_indices=[1]))
        gradient_penalty = tf.reduce_mean((slopes-1.)**2)

        return self.g_loss_calc(), self.d_loss_calc(gradient_penalty)

    def train(self):
        # Optimizer

        #learning_rate = 1e-4
        beta1 = 0.5
        beta2 = 0.9
       
        d_opt = tf.train.AdamOptimizer(learning_rate=self.learning_rate, beta1=beta1, beta2=beta2)
        d_train_op = d_opt.minimize(self.L_d, var_list=self.d.variables)
        g_opt = tf.train.AdamOptimizer(learning_rate=self.learning_rate, beta1=beta1, beta2=beta2)
        g_train_op = g_opt.minimize(self.L_g, var_list=self.g.variables)

        saver = tf.train.Saver()
        #%debug

        config = tf.ConfigProto(
            gpu_options=tf.GPUOptions(
                visible_device_list="0"
            )
        )
        
        with tf.Session(config=config) as sess:
            sess.run(tf.global_variables_initializer())

            lr = 1e-4
            for epoch in range(self.epochs):
                if epoch > 2000:
                    lr = 5e-5
                if epoch > 4000:
                    lr = 1e-5

                if epoch % self.epoch_saveSampleImg == 0:
                    imgs_gen = self.sample_images(inputs=self.noise).eval() 
                    #imgs_gen += 0.5
                    imgs_gen *= 1
                    print("saving images")
                    path = "generated_figures_gpu0"
                    save_imgs(path, imgs_gen, epoch=epoch)

                for _ in range(10):

                    # 訓練データを抜粋
                    Z_mb = np.random.uniform(-1, 1, size=[self.batch_size, self.z_dim]).astype(np.float32)

                    X_mb = extractXimg("./128x128x3resized_faceimgs", self.batch_size, self.img_size)
                    X_mb = X_mb.reshape([self.batch_size, -1])
                    X_mb = X_mb/255
                    X_p = X_mb + 0.5*np.std(X_mb)*np.random.random(X_mb.shape)

                    _, d_loss_value = sess.run([d_train_op, self.L_d], feed_dict={self.Z:Z_mb , self.X_tr: X_mb, self.X_p:X_p, self.learning_rate:lr})
                _, g_loss_value, = sess.run([g_train_op, self.L_g], feed_dict={self.Z:Z_mb, self.X_tr: X_mb, self.X_p:X_p, self.learning_rate:lr})

                # 結果をappend
                self.losses["d_loss"].append(d_loss_value)
                self.losses["g_loss"].append(g_loss_value)

                # グラフの描画（余裕があったら）
                if epoch % self.epoch_saveMetrics == 0:
                    print("epoch:" + str(epoch) + ", d_loss:" + str(d_loss_value) + ", g_loss:" + str(g_loss_value))
                    path = "metrics_gpu0"
                    save_metrics(path, self.losses, epoch)

                if epoch % self.epoch_saveParameter == 0:
                    path = "model_gpu0"
                    if not os.path.isdir(path):
                        os.makedirs(path)
                    saver.save(sess, path + "/dcgan_model" + str(epoch) + ".ckpt")

    def sample_images(self, row=5, col=12, inputs=None, epoch=None):
        images = self.g(inputs, training=True)
        return images

In [ ]:
gan = DRAGAN()
gan.train()

/home/ujitoko/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:72: DeprecationWarning: PyUnicode_AsEncodedObject() is deprecated; use PyUnicode_AsEncodedString() to encode from str to bytes or PyCodec_Encode() for generic encoding


saving images
epoch:0, d_loss:1.54122, g_loss:0.784011
saving images
epoch:100, d_loss:0.498339, g_loss:1.50246
